# Twitter Sentiment Analysis Stock Price Prediction

### - Crawl data

### - Sentiment Analysis

### - Predict Stock

## Import module

In [5]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install tweepy==3.8.0


from textblob import TextBlob
import math
from collections import Counter, defaultdict
import tweepy as tw
import json
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
%matplotlib inline
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score

  Attempting uninstall: tweepy
    Found existing installation: tweepy 4.5.0
    Uninstalling tweepy-4.5.0:
      Successfully uninstalled tweepy-4.5.0


## Set up token API

In [6]:
api_key = "HAcIeikl6eRheQP1oKYGDwPx0"
api_secret = "gEWmqG7QVXzudnviUXDo98L2UKGYg9PNqy1bAR1geClkHjqBGk"
access_token = "1481095074465579008-1pApYlU4HRJWpbAXcbdVEVGzdACuEz"
access_token_secret = "htgD81H4mOxXw5MglAljFRHhcfGAuwsNomXVqvGw0Obwe"
auth = tw.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth,wait_on_rate_limit=True)

## Crawl data

In [7]:
def apiSearch(kw, number_of_tweets, tweets, user_id, created_at):
    for i in tw.Cursor(api.search, q = kw , lang ="en", since = "2022-01-14",tweet_mode = "extended").items(number_of_tweets):
        tweets.append(i.full_text)
        user_id.append(i.user.id)
        created_at.append(i.created_at)

In [8]:
number_of_tweets = 100
tweets = []
user_id =[]
created_at = []
keyword_list = ['Finance', 'Shareholders', 'Stock', 'Soaring', 'Market', 'Trends']
company = 'Amazon'
final_kw_list = []

# Crawl data from keywords
for i in range (len(keyword_list)):
    kw = str(company + ' ' + keyword_list[i])
    print(kw)
    final_kw_list.append(kw)
    for i in tw.Cursor(api.search, q = kw , lang ="en", since = "2022-01-14",tweet_mode = "extended").items(number_of_tweets):
        tweets.append(i.full_text)
        user_id.append(i.user.id)
        created_at.append(i.created_at)



Amazon Finance


AttributeError: 'API' object has no attribute 'search'

## Dataframe

In [ ]:
df = pd.DataFrame({"Created_at": created_at,"User_id": user_id,"Tweets":tweets})
df

## Clean data

In [ ]:
df = df[~df.Tweets.str.contains("RT")]
df = df.reset_index(drop = True)

In [ ]:
def cleanUpTweet(text):
    text = re.sub(r'@[A-Za-z0-9_]+','',text) # Remove mentions @
    text = re.sub(r'#', '', text) # Remove hastag symbol
    text = re.sub(r'https?:\/\/[A-Za-z0-9\.\/]+', '', text) # Remove hyper link
    text = re.sub('[^A-Za-z0-9]+', ' ', text) # Remove special characters
    #Remove stop words
    res = ''
    for word in text.split():
        if word not in stopwords.words('english'):
            res += WordNetLemmatizer().lemmatize(word) + ' '
            
    return res

In [ ]:
df['Tweets'] = df['Tweets'].apply(cleanUpTweet)
df = df.drop_duplicates(subset='Tweets')
df = df.reset_index(drop=True)
df

## Create clean data list

In [ ]:
cleaned_data=[]
for tw in df['Tweets']:
    cleaned_data.append(tw)
    
cleaned_data

## WordCloud Visualization

In [ ]:
# Plot the word cloud 
allWords = ' '.join( [twts for twts in df['Tweets']] )
positiveWordCloud = WordCloud(max_words = 1000 , width = 1600 , height = 800, collocations=False).generate(allWords)
plt.figure(figsize = (15,15))
plt.imshow(positiveWordCloud, interpolation="bilinear")
plt.axis('off')
plt.show()


## Sentiment Analysis

In [ ]:
# Create function to get subjectivity
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

# Create function to get polarity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

# Function compute negative, positive
def getAnalysis(score):
    if score < 0: 
        return np.int_(0)
    elif score > 0:
        return np.int_(1)
    else:
        return 'Neutral'
    
df['Subjectivity'] = df['Tweets'].apply(getSubjectivity)
df['Polarity'] = df['Tweets'].apply(getPolarity)
df['Sentiment'] = df['Polarity'].apply(getAnalysis)
df.drop(df.loc[df['Sentiment']=='Neutral'].index, inplace=True)
df = df.reset_index(drop=True)
df

In [ ]:
# Save as CSV file
df.to_csv('Amazon.csv')

In [ ]:
# Display 5 positive tweets
j = 0
sortedDF = df.sort_values(by=['Polarity'])
positiveTweets = ''
for i in range(sortedDF.shape[0]):
    if (sortedDF['Sentiment'][i] == 1):
        print(str(j) + ') ' + sortedDF['Tweets'][i])
        positiveTweets += (' ' + sortedDF['Tweets'][i])
        print()
        j+=1
    if j == 5:
        break



In [ ]:
# Display 5 negative tweets
j = 0
sortedDF = df.sort_values(by=['Polarity'])
negativeTweets = ''
for i in range(sortedDF.shape[0]):
    if (sortedDF['Sentiment'][i] == 0):
        print(str(j) + ') ' + sortedDF['Tweets'][i] )
        negativeTweets += (' ' + sortedDF['Tweets'][i])
        print()
        j+=1
    if j == 5:
        break

In [ ]:
# Plot popularity and subjectivity
plt.figure(figsize=(8,6))
for i in range(df.shape[0]):
    plt.scatter(df['Polarity'][i], df['Subjectivity'][i], color='Blue')
    
plt.title('Sentiment Analysis')
plt.xlabel('Polarity')
plt.ylabel('Subjectivity')
plt.grid(True)
plt.show()

## Train model

In [ ]:
class Tokenizer:
  
  def clean(self, text):
      no_html = BeautifulSoup(text).get_text()
      clean = re.sub("[^a-z\s]+", " ", no_html, flags=re.IGNORECASE)
      return re.sub("(\s+)", " ", clean)

 
  def tokenize(self, text):
      clean = self.clean(text).lower()
      stopwords_en = stopwords.words("english")
      return [w for w in re.split("\W+", clean) if not w in stopwords_en]

In [ ]:
class MultinomialNaiveBayes:
  
    def __init__(self, classes, tokenizer):
      self.tokenizer = tokenizer
      self.classes = classes
      
    def group_by_class(self, X, y):
      data = dict()
      for c in self.classes:
        data[c] = X[np.where(y == c)]
      return data
           
    def fit(self, X, y):
        self.n_class_items = {}
        self.log_class_priors = {}
        self.word_counts = {}
        self.vocab = set()

        n = len(X)
        
        grouped_data = self.group_by_class(X, y)
        
        for c, data in grouped_data.items():
          self.n_class_items[c] = len(data)
          self.log_class_priors[c] = math.log(self.n_class_items[c] / n)
          self.word_counts[c] = defaultdict(lambda: 0)
          
          for text in data:
            counts = Counter(self.tokenizer.tokenize(text))
            for word, count in counts.items():
                if word not in self.vocab:
                    self.vocab.add(word)

                self.word_counts[c][word] += count
                
        return self
      
    def laplace_smoothing(self, word, text_class):
      num = self.word_counts[text_class][word] + 1
      denom = self.n_class_items[text_class] + len(self.vocab)
      return math.log(num / denom)
      
    def predict(self, X):
        result = []
        for text in X:
          
          class_scores = {c: self.log_class_priors[c] for c in self.classes}

          words = set(self.tokenizer.tokenize(text))
          for word in words:
              if word not in self.vocab: continue

              for c in self.classes:
                
                log_w_given_c = self.laplace_smoothing(word, c)
                class_scores[c] += log_w_given_c
                
          result.append(max(class_scores, key=class_scores.get))

        return result

In [ ]:
X = df['Tweets'].values
y = df['Sentiment'].values
RANDOM_SEED = 1
  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
MNB = MultinomialNaiveBayes(
    classes=np.unique(y), 
    tokenizer=Tokenizer()
).fit(X_train, y_train)


In [ ]:
y_hat = np.array(MNB.predict(X_test))
print(y_test)
print(y_hat)
print(type(y_hat[0]))
print(type(y_test[0]))

In [ ]:
accuracy_score(y_test, y_hat)

In [ ]:
print(classification_report(y_test, y_hat))

In [ ]:
cnf_matrix = confusion_matrix(y_test, y_hat)
cnf_matrix

In [ ]:
class_names = ["negative", "positive"]
fig,ax = plt.subplots()


sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="Blues", fmt="d", cbar=False, xticklabels=class_names, yticklabels=class_names)
ax.xaxis.set_label_position('top')
plt.tight_layout()
plt.ylabel('Actual sentiment')
plt.xlabel('Predicted sentiment');